In [1]:
import glob
import os
from google.oauth2.service_account import Credentials
os.environ.get('SECRETS_PATH')
with open('/langley/udocker/phoenix-worker/current/phoenix-worker/meehir-dsw-gcp-secret/creds') as f:
    
    secret_file = f.read()
secret = eval(secret_file)
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive',
]
creds = Credentials.from_service_account_info(info=secret,scopes=scopes)

In [2]:
import pandas as pd
import numpy as np
import os
import glob
from querybuilder_client import QuerybuilderClient
from queryrunner_client import Client
import datetime
import base64
from IPython.display import HTML
import warnings
import pygsheets
import ast
import IPython
from numpy import nan
import json
# import pypostmaster
warnings.filterwarnings('ignore')

from dateutil.parser import parse
from time import sleep
from datetime import datetime
from datetime import timedelta

import os
from oauth2client.service_account import ServiceAccountCredentials

qr = Client(user_email='mrajor@ext.uber.com', consumer_name='marketplace-mobility-ops-analytics')

gc = pygsheets.authorize(custom_credentials=creds)

wsaccess = pygsheets.authorize(custom_credentials=creds)
main_sheet = gc.open_by_key('1td9RLTlnKFbKGEy9mDCmerz2oZUCj-tRhc7ufch0uyY')
details = main_sheet.worksheet_by_title('manually_handled_npi')
qr = Client(user_email='mrajor@ext.uber.com', consumer_name='marketplace-mobility-ops-analytics')

01/20/2025 02:55:32 PM  Obtained utoken email from file 
01/20/2025 02:55:33 PM  Obtained utoken from utoken cli 
01/20/2025 02:55:33 PM  Obtained utoken email from file 
01/20/2025 02:55:33 PM  Obtained utoken from utoken cli 


In [3]:
sheet=details.get_as_df()
sheet

,Report,tab_name,sheet,queries
0,FqrD0rZqX,Uber_raw_data,https://docs.google.com/spreadsheets/d/1td9RLT...,https://querybuilder.uberinternal.com/r/FqrD0r...
1,84DjV3IUH,ola_raw_data,https://docs.google.com/spreadsheets/d/1td9RLT...,https://querybuilder.uberinternal.com/r/84DjV3...
2,FqrD0rZqX,Uber_raw_data,https://docs.google.com/spreadsheets/d/12D5K2c...,https://querybuilder.uberinternal.com/r/FqrD0r...
3,dIpNNhMGL,rapido_raw_data,https://docs.google.com/spreadsheets/d/12D5K2c...,https://querybuilder.uberinternal.com/r/dIpNNh...


In [ ]:
today=datetime.today()
start_date=today-timedelta(days=1)
end_date=today-timedelta(days=0)
start=start_date.strftime("%Y-%m-%d")
end=end_date.strftime("%Y-%m-%d")
print(start)
print(end)

In [5]:
week_number=start_date.isocalendar()[1]
print(week_number)

3


In [6]:
def upload_in_wks_replace(dash_link,tab_name,df):
    wks=gc.open_by_url(dash_link).worksheet_by_title(tab_name)
    wks.clear()
    lastrow=len(wks.get_col(1,include_tailing_empty=False)) + 1
    wks.set_dataframe(df,start='A{}'.format(lastrow),copy_head=False)
    print("data uploaded")

In [7]:
def upload_in_wks(dash_link,tab_name,df):
    wks=gc.open_by_url(dash_link).worksheet_by_title(tab_name)
    wks.add_rows(df.shape[0])
    lastrow=len(wks.get_col(1,include_tailing_empty=False)) + 1
    wks.set_dataframe(df,start='A{}'.format(lastrow),copy_head=False)
    print("data uploaded")

In [8]:
def make_data_types_common(df):
    df['day'] = df['day'].astype(str)
    df['Time'] = df['Time'].astype(str)
    df['city_name'] = df['city_name'].astype(str)
    df['scr_geo'] = df['scr_geo'].astype(str)
    df['dest_geo'] = df['dest_geo'].astype(str)
    df['Distance_bucket'] = df['Distance_bucket'].astype(str)
#     df['week'] = df['week'].astype(str)
    return df

In [9]:
def merging_df(uber_df, df):
    print("Uber DataFrame Columns:", uber_df.columns.tolist())
    print("Input DataFrame Columns:", df.columns.tolist())
    
    # Ensure 'day' is in datetime format
    df['day'] = pd.to_datetime(df['day'])
    df['day'] = df['day'].dt.strftime('%Y-%m-%d')

    # Ensure numeric types for relevant columns, check if they exist
    numeric_columns = ['sum_fare', 'sum_dist', 'points', 'requests', 'req weighted dist']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if col in uber_df.columns:
            uber_df[col] = pd.to_numeric(uber_df[col], errors='coerce')

    # Merge DataFrames
    merged_df = pd.merge(uber_df, df, on=['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'], how='inner')

    # Group and transform
    if 'sum_fare' in merged_df.columns:
        merged_df['sum_fare'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_fare'].transform('sum')
    if 'sum_dist' in merged_df.columns:
        merged_df['sum_dist'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_dist'].transform('sum')
    if 'points' in merged_df.columns:
        merged_df['points'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['points'].transform('sum')

    merged_df['day'] = pd.to_datetime(merged_df['day'])
    merged_df['week'] = merged_df['day'].dt.isocalendar().week

    if 'requests' in merged_df.columns:
        merged_df['requests'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['requests'].transform('sum')
    if 'req weighted dist' in merged_df.columns:
        merged_df['uber_dist'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['req weighted dist'].transform('sum')

    # Convert requests to int
    if 'requests' in merged_df.columns:
        merged_df['requests'] = merged_df['requests'].astype(int)

    # Calculate weighted fare only if columns exist
    if 'sum_fare' in merged_df.columns and 'sum_dist' in merged_df.columns and 'uber_dist' in merged_df.columns:
        merged_df['req weighted fare'] = (merged_df['sum_fare'] / merged_df['sum_dist']) * merged_df['uber_dist']
    if 'requests' in merged_df.columns and 'sum_dist' in merged_df.columns:
        merged_df['req weighted dist'] = merged_df['requests'] * merged_df['sum_dist']

    return merged_df


In [10]:
def merging_df_rapido(uber_df, df):
    print("Rapido Uber DataFrame Columns:", uber_df.columns.tolist())
    print("Input DataFrame Columns:", df.columns.tolist())
    
    df['day'] = pd.to_datetime(df['day'])
    df['day'] = df['day'].dt.strftime('%Y-%m-%d')

    # Ensure numeric types for relevant columns, check if they exist
    numeric_columns = ['sum_fare_pre', 'sum_fare_post', 'sum_dist', 'points', 'requests', 'req weighted dist']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if col in uber_df.columns:
            uber_df[col] = pd.to_numeric(uber_df[col], errors='coerce')

    merged_df = pd.merge(uber_df, df, on=['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'], how='inner')

    if 'sum_fare_pre' in merged_df.columns:
        merged_df['sum_fare_pre'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_fare_pre'].transform('sum')
    if 'sum_fare_post' in merged_df.columns:
        merged_df['sum_fare_post'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_fare_post'].transform('sum')
    if 'sum_dist' in merged_df.columns:
        merged_df['sum_dist'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_dist'].transform('sum')
    if 'points' in merged_df.columns:
        merged_df['points'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['points'].transform('sum')

    #merged_df['week'] = merged_df['day'].dt.isocalendar().week
    merged_df['week'] = week_number

    if 'requests' in merged_df.columns:
        merged_df['requests'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['requests'].transform('sum')
    if 'req weighted dist' in merged_df.columns:
        merged_df['uber_dist'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['req weighted dist'].transform('sum')

    if 'requests' in merged_df.columns:
        merged_df['requests'] = merged_df['requests'].astype(int)

    if 'sum_fare_pre' in merged_df.columns and 'sum_dist' in merged_df.columns and 'uber_dist' in merged_df.columns:
        merged_df['req weighted fare_pre'] = (merged_df['sum_fare_pre'] / merged_df['sum_dist']) * merged_df['uber_dist']
    if 'sum_fare_post' in merged_df.columns and 'sum_dist' in merged_df.columns and 'uber_dist' in merged_df.columns:
        merged_df['req weighted fare_post'] = (merged_df['sum_fare_post'] / merged_df['sum_dist']) * merged_df['uber_dist']
    
    if 'requests' in merged_df.columns and 'sum_dist' in merged_df.columns:
        merged_df['req weighted dist'] = merged_df['requests'] * merged_df['sum_dist']

    return merged_df


In [ ]:
# Main Logic
ola_uber_df = pd.DataFrame()
app_df = pd.DataFrame()
rapido_uber_df = pd.DataFrame()
moto_uber_df_web = pd.DataFrame()
moto_uber_df_app = pd.DataFrame()

for i in range(0,4):
    df = qr.execute_report(sheet.iloc[i, 0], parameters={"start": start, "end": end}, datacenter='phx2').to_pandas()
#     try:
#         df = qr.execute_report(sheet.iloc[i, 0], parameters={"start": start, "end": end}, datacenter='phx2').to_pandas()
    
#     # Add delay bw iteration
#         if i <4: # don't sleep after last iteration
#             time.sleep(60)
#     except Exception as e:
#         if "DuplicateQuery" in str(e):
#             time.sleep(10) #wait 10 seconds before retry
#             df = qr.execute_report(sheet.iloc[i, 0], parameters={"start": start, "end": end}, datacenter='phx2').to_pandas()
#         raise e
    if sheet.iloc[i, 1] == 'Uber_raw_data':
        df['day'] = pd.to_datetime(df['day'])
        df['week'] = df['day'].dt.isocalendar().week
        df['req weighted fare'] = df['sum_fare']
        df['req weighted dist'] = df['sum_dist']
        ola_uber_df = df
        rapido_uber_df = df
        app_df = df
        upload_in_wks(sheet.iloc[i, 2], sheet.iloc[i, 1], df) 

        
    elif sheet.iloc[i, 1] == 'ola_raw_data':
        print(app_df)
        app_df.drop(columns=['sum_fare', 'sum_dist', 'points', 'req weighted fare', 'week'], inplace=True)
        app_df = make_data_types_common(app_df)
        df = make_data_types_common(df)
        new_df = merging_df(app_df, df)
        temp_df = new_df[['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket',
                           'sum_fare', 'sum_dist', 'points', 'week', 'requests',
                           'req weighted fare', 'uber_dist', 'req weighted dist']]
        upload_in_wks(sheet.iloc[i, 2], sheet.iloc[i, 1], temp_df)  
        
    elif sheet.iloc[i,1]=='rapido_raw_data':
        print(rapido_uber_df.columns)
        rapido_uber_df=rapido_uber_df[['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket',
       'requests', 'req weighted dist']]
        rapido_uber_df=make_data_types_common(rapido_uber_df)
        df=make_data_types_common(df)
        rapido_new_df=merging_df_rapido(rapido_uber_df,df)
        rapido_temp_df = rapido_new_df[['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket',
       'sum_fare_pre','sum_fare_post', 'sum_dist', 'points', 'week', 'requests',
       'req weighted fare_pre','req weighted fare_post', 'uber_dist', 'req weighted dist']]
        upload_in_wks(sheet.iloc[i,2],sheet.iloc[i,1],rapido_temp_df)  



01/20/2025 02:55:34 PM  Could not obtain utoken from cli: b'/bin/sh: 1: usso: not found\n'
Trying to obtain from file 
01/20/2025 02:55:34 PM  Obtained utoken for user 
01/20/2025 02:55:34 PM  Fetching metadata for Report FqrD0rZqX 
01/20/2025 02:55:34 PM  Loaded object metadata. 
01/20/2025 02:55:34 PM  Templating query for report FqrD0rZqX 
01/20/2025 02:55:34 PM  Templated query successfully. 
01/20/2025 02:55:34 PM Send tier_metadata {'report_id': 'FqrD0rZqX'} to Queryrunner V2.
01/20/2025 02:55:34 PM  [Polling] 036d4d71-4311-4d2a-965a-d92518b18853 
01/20/2025 02:55:34 PM  [Status] created 
01/20/2025 02:55:35 PM  [Status] started validation 
01/20/2025 02:55:39 PM  [Status] started auth check 
01/20/2025 02:55:43 PM  [Status] started waiting to execute 
01/20/2025 03:00:20 PM  UToken has expired. Generating new utoken. 
01/20/2025 03:00:20 PM  Obtained utoken email from file 
01/20/2025 03:00:20 PM  Obtained utoken from utoken cli 
01/20/2025 03:03:20 PM  UToken has expired. Gener